In [1]:
import uploader
import psycopg2
import folium
import pandas as pd

In [2]:
loader = uploader.TrafficDatabaseLoader('psql_config.json')
loader.import_from_osm('/home/axer/Downloads/map.osm')

In [3]:
conn = psycopg2.connect(dbname='traffic', user='postgres', 
                        password='1234', host='localhost')

cursor = conn.cursor()

cursor.execute("SELECT * FROM Node WHERE id in (SELECT node_id FROM Node_tag WHERE k LIKE 'highway')")
records = cursor.fetchall()
df = pd.DataFrame(records)

cursor.close()
conn.close()

In [4]:
df.rename(columns={0:'id', 1:'lat', 2:'lon'}, inplace=True)
df.sample(10)

,id,lat,lon
15,4144042567,55.732931,37.606465
23,4144073956,55.733367,37.605270
31,4144042568,55.733035,37.607119
47,1252963235,55.735488,37.613100
21,4144073955,55.733260,37.605071
19,2546149017,55.733300,37.615291
10,1431285766,55.733113,37.609540
46,4144042564,55.732715,37.606936
14,1431285738,55.732498,37.609266
45,4144043690,55.733616,37.607706


In [5]:
def show_circles_on_map(data, latitude_column, longitude_column, color):
    """
    The function draws map with circles on it.
    The center of the map is the mean of coordinates passed in data.
    
    data: DataFrame that contains columns latitude_column and longitude_column
    latitude_column: string, the name of column for latitude coordinates
    longitude_column: string, the name of column for longitude coordinates
    color: string, the color of circles to be drawn
    """

    location = (data[latitude_column].mean(), data[longitude_column].mean())
    m = folium.Map(location=location, zoom_start=99)

    for _, row in data.iterrows():
        folium.Circle(
            radius=1,
            location=(row[latitude_column], row[longitude_column]),
            color=color,
            fill_color=color,
            fill=True
        ).add_to(m)

    return m

In [6]:
show_circles_on_map(df, 'lat', 'lon', 'blue')